In [1]:
from keras.utils import plot_model
import threading
import time
# INITIALIZATION: libraries, parameters, network...

from keras.models import Sequential      # One layer after the other
from keras.layers import Dense, Flatten  # Dense layers are fully connected layers, Flatten layers flatten out multidimensional inputs
from collections import deque            # For storing moves 
import World
import numpy as np
import gym                                # To train our network

import tensorflow as tf
import random

Using TensorFlow backend.


In [2]:
# For sampling batches from the observations


    # Parameters
D = deque()                                # Register where the actions will be stored

observetime = 300                          # Number of timesteps we will be acting on the game and observing results
epsilon = 0.9                              # Probability of doing a random move
gamma = 0.9                                # Discounted future reward. How much we care about steps further in time
mb_size = 100     
input_p = tf.placeholder(tf.float32, [None,5])

q_target = tf.placeholder(tf.float32, [None ,4], name='Q_target')
with tf.variable_scope('l1'):
        w1 = tf.get_variable('w1', [5,15], initializer=tf.random_uniform_initializer())
        b1 = tf.get_variable('b1', [1,15], initializer=tf.zeros_initializer())
        l1 = tf.nn.relu(tf.matmul(input_p,w1 ) + b1)
with tf.variable_scope('l2'):
        w2 = tf.get_variable('w2', [15, 15], initializer=tf.random_uniform_initializer())
        b2 = tf.get_variable('b2', [1, 15], initializer=tf.zeros_initializer())
        l2 = tf.nn.relu(tf.matmul(l1,w2) + b2)

    # second layer. collections is used later when assign to target net
with tf.variable_scope('l3'):
        w3 = tf.get_variable('w3', [15,4], initializer=tf.random_uniform_initializer())
        b3 = tf.get_variable('b3', [1,4], initializer=tf.zeros_initializer())
        Q = (tf.matmul(l2,w3) + b3)
with tf.variable_scope('loss'):
        sqd=tf.squared_difference(q_target, Q)
        loss = tf.reduce_mean(sqd)       
trainer = tf.train.RMSPropOptimizer(learning_rate=0.01)
updateModel = trainer.minimize(loss)        
# model = Sequential()
# model.add(Dense(10, input_shape=(5,) , init='uniform', activation='relu'))
#     #model.add(Flatten())       # Flatten input so as to have no problems with processing
# model.add(Dense(10, init='uniform', activation='relu'))
# model.add(Dense(4, init='uniform', activation='linear'))    # Same number of outputs as possible actions

# model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
# plot_model(model, to_file='model.png',show_shapes=True)
                          # Learning minibatch size

In [3]:
def do_action(action):
    #r = -World.score
    if action == 0:
        rew,s,s1,res=World.try_move(0, -1)
    elif action == 1:
        rew,s,s1,res=World.try_move(0, 1)
    elif action == 2:
        rew,s,s1,res=World.try_move(-1, 0)
    elif action == 3:
        rew,s,s1,res=World.try_move(1, 0)
    #r += World.score
    return rew,s,s1,res

In [4]:
# FIRST STEP: Knowing what each action does (Observing)
def observe(sess):
    done = False
    state=World.findState()
    for t in range(observetime):
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, 4)
        else:
            Qs = sess.run(Q,feed_dict={input_p:state})          # Q-values predictions
            action = np.argmax(Qs,axis=1)[0]             # Move with highest Q-value is the chosen one
        reward,state,state_new,res = do_action(action)     # See state of the game, reward... after performing the action
                 # Update the input with the new state of the game
        D.append((state, action, reward, state_new, res))
            
    print('Observing Finished')

In [5]:
# SECOND STEP: Learning from the observations (Experience replay)

                            # Sample some moves

def learn(sess):
    inputs = np.zeros((mb_size,5))
    targets = np.zeros((mb_size, 4))
    minibatch = random.sample(D, mb_size) 
    for i in range(mb_size):
            state = minibatch[i][0]
            action = minibatch[i][1]
            reward = minibatch[i][2]
            state_new = minibatch[i][3]
            done = minibatch[i][4]

    # Build Bellman equation for the Q function
            inputs[i:i+1] = state   # Q-values predictions

            targets[i] = sess.run(Q,feed_dict={input_p:state})
            Q_sa = sess.run(Q,feed_dict={input_p:state_new})
            if done:
                targets[i, action] = reward
            else:
                targets[i, action] = reward + gamma * np.max(Q_sa,axis=1)
    a,b=sess.run([updateModel,loss],feed_dict={input_p:inputs,q_target:targets})
    print("Cost",b)
        
    # Train network to output the Q function
    D.clear()
       
    print('Learning Finished')
    #print(inputs,targets)

In [7]:
# THIRD STEP: Play!
def test(sess):
    print("Evaluating")
    s1 = World.findState()
    done = False
    tot_reward = 0.0
    i=0
    while i<50:                 # Uncomment to see game running
        Qs = sess.run(Q,feed_dict={input_p:s1})
        action = np.argmax(Qs,axis=1)   
        #print(Q,action)
        rew,s,s1,res = do_action(action[0]) 
        tot_reward += rew
        print('Game : ',action,"   reward ",rew,"state",s,res)
        i+=1
        if res:
            break
    print('Game ended! Total reward: {}'.format(tot_reward))

In [8]:
def process():
    global epsilon
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for i in range(1000):
            World.restart_game()
            observe(sess)
            learn(sess)
            test(sess)
            epsilon -=0.03                              
process()

Observing Finished
Cost 36576.5
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  1.          10.          13.           1.          13.03840446]] False
Game :  [0]    reward  0 state [[  1.           9.          13.           2.          12.04159451]] False
Game :  [0]    reward  0 state [[  1.           8.          13.           3.          11.04536057]] False
Game :  [0]    reward  0 state [[  1.           7.          13.           4.          10.04987526]] False
Game :  [0]    reward  0 state [[  1.           6.          13.           5.           9.05538559]] False
Game :  [0]    reward  0 state [[ 1.          5.          6.          6.          8.06225777]] False
Game :  [0]    reward  0 state [[ 1.          4.          5.          7.          7.07106781]] False
Game :  [0]    reward  0 state [[ 1.          3.          2.          8.          6.08276272]] False
Game :  [0]    reward  0 state [[  1.           2.          13.           9.           5.09901953]] False


Cost 5891.72
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  1.           7.          13.           4.          10.04987526]] False
Game :  [0]    reward  0 state [[  1.           6.          13.           5.           9.05538559]] False
Game :  [0]    reward  0 state [[ 1.          5.          6.          6.          8.06225777]] False
Game :  [0]    reward  0 state [[ 1.          4.          5.          7.          7.07106781]] False
Game :  [0]    reward  0 state [[ 1.          3.          2.          8.          6.08276272]] False
Game :  [0]    reward  0 state [[  1.           2.          13.           9.           5.09901953]] False
Game :  [0]    reward  0 state [[  1.           1.          13.          10.           4.12310553]] False
Game :  [0]    reward  -0.5 state [[  1.          0.          1.         11.          3.1622777]] True
Game ended! Total reward: -0.5
Observing Finished
Cost 6262.64
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  0

Cost 1649.57
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  0.  13.  14.   1.  13.]] False
Game :  [0]    reward  0 state [[  0.  12.  14.   2.  12.]] False
Game :  [3]    reward  0 state [[  0.  11.  14.   3.  11.]] False
Game :  [3]    reward  0 state [[  1.           8.          13.           3.          11.04536057]] False
Game :  [0]    reward  0 state [[  2.           8.          12.           3.          11.18033981]] False
Game :  [3]    reward  0 state [[  2.           7.          12.           4.          10.19803905]] False
Game :  [3]    reward  0 state [[  3.           6.          11.           4.          10.44030666]] False
Game :  [3]    reward  0 state [[  4.           3.          10.           4.          10.77032948]] False
Game :  [0]    reward  0 state [[  5.          10.           9.           4.          11.18033981]] False
Game :  [0]    reward  0 state [[  5.           9.           9.           5.          10.29563046]] False
Game :  [0]    rew

Game :  [3]    reward  -0.5 state [[ 14.   0.   0.  14.  14.]] True
Game ended! Total reward: -0.5
Observing Finished
Cost 761.16
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  1.          11.          13.           0.          14.03566837]] False
Game :  [0]    reward  0 state [[  1.          10.          13.           1.          13.03840446]] False
Game :  [0]    reward  0 state [[  1.           9.          13.           2.          12.04159451]] False
Game :  [0]    reward  0 state [[  1.           8.          13.           3.          11.04536057]] False
Game :  [0]    reward  0 state [[  1.           7.          13.           4.          10.04987526]] False
Game :  [3]    reward  0 state [[  1.           6.          13.           5.           9.05538559]] False
Game :  [0]    reward  0 state [[  2.           6.          12.           5.           9.21954441]] False
Game :  [0]    reward  0 state [[ 2.          5.          5.          6.          8.24621105]] Fals

Observing Finished
Cost 235.876
Learning Finished
Evaluating
Game :  [1]    reward  0 state [[ 3.         3.         3.         7.         7.6157732]] False
Game :  [1]    reward  0 state [[ 3.          4.          4.          6.          8.54400349]] False
Game :  [3]    reward  0 state [[  3.           5.          11.           5.           9.48683262]] False
Game :  [3]    reward  0 state [[  4.           2.          10.           5.           9.84885788]] False
Game :  [1]    reward  0 state [[  5.           9.           9.           5.          10.29563046]] False
Game :  [1]    reward  0 state [[  5.          10.           9.           4.          11.18033981]] False
Game :  [1]    reward  0 state [[  5.          11.           9.           3.          12.08304596]] False
Game :  [1]    reward  0 state [[  5.  12.   9.   2.  13.]] False
Game :  [1]    reward  0 state [[  5.         13.          9.          1.         13.9283886]] False
Game :  [0]    reward  0 state [[  5.        

...

Observing Finished
Cost 0.0018896
Learning Finished
Evaluating
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   4.  14.  10.   4.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  6.  3.  8.  6.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  6.  3.  8.  6.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  6.  3.  8.  6.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  6.  3.  8.  6.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  6.  3.  8.  6.]] False
Game :  [1]    reward  0 state [[  0.   5.  14.   9.   5.]] False
Game :  [0]    reward  0 state [[ 0.  

Observing Finished
Cost 0.00220411
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [1]    reward  0 state [[  0.   1.   0.  13.   1.]] False
Game :  [0] 

Observing Finished
Cost 0.000379224
Learning Finished
Evaluating
Game :  [0]    reward  0 state [[  0.   4.  14.  10.   4.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]    reward  0 state [[  0.   2.   0.  12.   2.]] False
Game :  [0]    reward  0 state [[  0.   3.   2.  11.   3.]] False
Game :  [1]

Observing Finished


KeyboardInterrupt: 